In [40]:
import  torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.utils.data as td
import torch.nn.functional as F

定义变量

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_works = 4
batch_size = 256
input_dim=100
epochs = 1
print("run on ",device.type)

run on  cpu


In [42]:
from typing import Iterable


# loading data
def loading_minist_data(batch_siz:int, works:int)->(Iterable,Iterable):

    train_set = torchvision.datasets.MNIST(root="./data",train=True,transform=torchvision.transforms.ToTensor(),download=True)
    test_set = torchvision.datasets.MNIST(root="./data",train=False,transform=torchvision.transforms.ToTensor(),download=True)
    return td.DataLoader(dataset=train_set, batch_size=batch_siz, shuffle=True, num_workers=works,drop_last=True), td.DataLoader(dataset=test_set, batch_size=batch_siz, shuffle=True, num_workers=works,drop_last=True)


In [47]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            # 28*28 -> (28 )/2 = 12*12
            nn.Conv2d(1, 32,kernel_size=3,padding=1, stride=2,dtype=torch.float32),
            nn.LeakyReLU(0.01),
            # 14,14 -> 7*7
            nn.Conv2d(32,64,kernel_size=3,padding=1,stride=2,dtype=torch.float32),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.01),
            # 10 filters to 10 filters
            # 7*7 -> 3*3
            nn.Conv2d(64, 128, kernel_size=3,stride=2,dtype=torch.float32),
            nn.LeakyReLU(0.02),
            nn.BatchNorm2d(128),
            nn.Flatten(),
            nn.Linear(128*3*3,10),
            # nn.ReLU()
            # nn.Linear(128,1),
            nn.Sigmoid()
        ).to(device)
        self.cnt=0
        self.loss_metric=[]
        self.loss_func = nn.BCELoss()
        self.optimizer = torch.optim.Adam(self.parameters())
        pass
    def forward(self,inputs):
        for i in self.model:
            # print(inputs.shape)
            inputs = i(inputs)
        return inputs
        # return self.model(inputs)
    def train(self,inputs,targets):
        out = self.forward(inputs)

        loss = self.loss_func(out,targets)
        self.cnt+=1
        self.loss_metric.append(loss.item())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        pass

class Generator(nn.Module):
    def __init__(self,in_dim:int):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(in_dim, 200),
            nn.LeakyReLU(0.02),
            nn.LayerNorm(200),
            nn.Linear(200, 784),
            nn.Sigmoid()
        ).to(device)
        self.optimiser = torch.optim.Adam(self.parameters(), lr=0.0001)
        # counter and accumulator for progress
        self.cnt = 0
        self.loss_metric = []


    def forward(self,inputs):
        return self.model(inputs).view(batch_size,28,28)
    def train(self,D:Discriminator,inputs,targets):
        gen_out = self.forward(inputs)
        # print(gen_out.shape)
        d_out = D.forward(gen_out)

        loss  = D.loss_func(d_out,targets)
                # zero gradients, perform a backward pass, update weights
        self.cnt+=1
        self.loss_metric.append(loss.item())
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()

def generate_random_seed(size):
    random_data = torch.randn(size*batch_size).view(batch_size,size)
    return random_data.to(device)


In [51]:
%%time
G = Generator(input_dim)
D = Discriminator()
train_iter ,test_iter = loading_minist_data(batch_size,4)
print("generate nums is run on ",generate_random_seed(input_dim).device)

for k in range(epochs):

    for it,y in train_iter:
    # print(y)
        out = D.forward(it)
    # y = torch.reshape(y,out.shape)
        loss = F.cross_entropy(out,y)
        print(loss.item())
    # print(loss)
        D.optimizer.zero_grad()
        loss.backward()
        D.optimizer.step()

# output = G.forward(generate_random_seed(input_dim))
# img = output.detach().numpy().reshape(28,28)
# plt.imshow(output.cpu().detach().numpy()[0], interpolation='none', cmap='Blues')

generate nums is run on  cpu
2.3368585109710693
2.1489381790161133
2.023455858230591
1.9598240852355957
1.8997929096221924
1.8605726957321167
1.8319669961929321
1.8174470663070679
1.761227011680603
1.7657605409622192
1.7399898767471313
1.7435168027877808
1.7090314626693726
1.6886435747146606
1.6749978065490723
1.6808104515075684
1.6753184795379639
1.6733266115188599
1.6503156423568726
1.646734356880188
1.6524083614349365
1.6214808225631714
1.6492966413497925
1.6436840295791626
1.6389973163604736
1.601409912109375
1.617828607559204
1.5979033708572388
1.5815285444259644
1.5955071449279785
1.5858880281448364
1.602893590927124
1.5752931833267212
1.5841491222381592
1.6161779165267944
1.578624963760376
1.6027278900146484
1.5805777311325073
1.60039484500885
1.572749137878418


KeyboardInterrupt: 

In [49]:
loss_metric = []
cnt=0
for it,y in test_iter:
    out = D.forward(it)
    loss = F.cross_entropy(out,y)
    cnt+=1
    if(cnt%10==0):
        loss_metric.append(loss.item())


In [50]:
# print(loss_metric)
# output = G.forward(generate_random_seed(input_dim))
# img = output.detach().numpy().reshape(28,28)
# plt.imshow(output.detach().numpy()[0], interpolation='none', cmap='Blues')

[1.492141842842102, 1.4767476320266724, 1.4831912517547607]
